<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
<ul class="toc-item">
<li><span><a href="#Pre-requisites" data-toc-modified-id="Pre-requisites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Pre-requisites</a></span></li>
<li><span><a href="#Instructions" data-toc-modified-id="Instructions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Instructions</a></span></li>
<li><span><a href="#Imports-and-Constants" data-toc-modified-id="Imports-and-Constants-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Imports and Constants</a></span></li>
<li><span><a href="#Constants" data-toc-modified-id="Constants-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Constants</a></span></li>
<li><span><a href="#Export-Images" data-toc-modified-id="Export-Images-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Export Images</a></span></li>
<li><span><a href="#Visualize-an-exported-image" data-toc-modified-id="Visualize-an-exported-image-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Visualize an exported image</a></span></li>
</ul>
</div>

## Pre-requisites
Register a Google account at [https://code.earthengine.google.com](https://code.earthengine.google.com). This process may take a couple of days. Without registration, the `ee.Initialize()` command below will throw an error message.

## Instructions

This notebook exports Landsat satellite image composites of DHS clusters from Google Earth Engine.

The images are saved in gzipped TFRecord format. By default, this notebook exports images to Google Drive. If you instead prefer to export images to Google Cloud Storage (GCS), change the `EXPORT` constant below to `'gcs'` and set `BUCKET` to the desired GCS bucket name.


|      | Storage  | Expected Export Time
|------|----------|---------------------
| DHS  | ~16.0 GB | ~24h

The exported images take up a significant amount of storage space. Before exporting, make sure you have enough storage space. The images are exported to the following locations, based on the constants `EXPORT` and `BUCKET` defined below:

|      | Google Drive (default) | GCS
|------|:-----------------------|:---
| DHS  | `dhs_tfrecords_raw/`   | `{BUCKET}/dhs_tfrecords_raw/`


Once the images have finished exporting, download the exported TFRecord files to the following folders:

- DHS: `data/dhs_tfrecords_raw/`


The folder structure should look as follows:

```
data/
    dhs_tfrecords_raw/
        angola_2011_00.tfrecord.gz
        ...
        zimbabwe_2015_00.tfrecord.gz

```

## Imports and Constants

In [15]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from typing import Any, Dict, Optional, Tuple
import ee, os, math
%load_ext autoreload
%autoreload 2

In [16]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/temporal-eo-wealth/download')
import ee_utils

Mounted at /content/drive


Before using the Earth Engine API, you must perform a one-time authentication that authorizes access to Earth Engine on behalf of your Google account you registered at [https://code.earthengine.google.com](https://code.earthengine.google.com). The authentication process saves a credentials file to `$HOME/.config/earthengine/credentials` for future use.

The following command `ee.Authenticate()` runs the authentication process. Once you successfully authenticate, you may comment out this command because you should not need to authenticate again in the future, unless you delete the credentials file. If you do not authenticate, the subsequent `ee.Initialize()` command below will fail.

For more information, see [https://developers.google.com/earth-engine/python_install-conda.html](https://developers.google.com/earth-engine/python_install-conda.html).

In [46]:
ee.Authenticate()

In [48]:
#ee.Initialize(project='ee-stevenndungu')  # initialize the Earth Engine API
ee.Initialize(project='ee-janet')

## Constants

In [18]:
# ========== ADAPT THESE PARAMETERS ==========

# To export to Google Drive, uncomment the next 2 lines
EXPORT = 'drive'
BUCKET = None

# To export to Google Cloud Storage (GCS), uncomment the next 2 lines
# and set the bucket to the desired bucket name
# EXPORT = 'gcs'
# BUCKET = 'mybucket'

# export location parameters
DHS_EXPORT_FOLDER = 'dhs_tfrecords_raw'



In [19]:
# ========== DO NOT MODIFY THESE ==========

# input data paths
DHS_CSV_PATH = '/content/drive/MyDrive/temporal-eo-wealth/data/dhs_clusters.csv'

# band names
MS_BANDS = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP1']

# image export parameters
PROJECTION = 'EPSG:3857'  # see https://epsg.io/3857
SCALE = 30                # export resolution: 30m/px
EXPORT_TILE_RADIUS = 127  # image dimension = (2*EXPORT_TILE_RADIUS) + 1 = 255px
CHUNK_SIZE = 1         # set to a small number (<= 50) if Google Earth Engine reports memory errors

## Export Images

In [6]:
def export_images(
        df: pd.DataFrame,
        country: str,
        year: int,
        export_folder: str,
        chunk_size: Optional[int] = None,
        ) -> Dict[Tuple[Any], ee.batch.Task]:
    '''
    Args
    - df: pd.DataFrame, contains columns ['lat', 'lon', 'country', 'year']
    - country: str, together with `year` determines the survey to export
    - year: int, together with `country` determines the survey to export
    - export_folder: str, name of folder for export
    - chunk_size: int, optionally set a limit to the # of images exported per TFRecord file
        - set to a small number (<= 50) if Google Earth Engine reports memory errors

    Returns: dict, maps task name tuple (export_folder, country, year, chunk) to ee.batch.Task
    '''
    subset_df = df[(df['country'] == country) & (df['year'] == year)].reset_index(drop=True)
    if chunk_size is None:
        num_chunks = 1
    else:
        num_chunks = int(math.ceil(len(subset_df) / chunk_size))
    tasks = {}

    for i in range(num_chunks):
        chunk_slice = slice(i * chunk_size, (i+1) * chunk_size - 1)  # df.loc[] is inclusive
        fc = ee_utils.df_to_fc(subset_df.loc[chunk_slice, :])
        start_date, end_date = ee_utils.surveyyear_to_range(year)

        # create 3-year Landsat composite image
        roi = fc.geometry()
        imgcol = ee_utils.LandsatSR(roi, start_date=start_date, end_date=end_date).merged
        imgcol = imgcol.map(ee_utils.mask_qaclear).select(MS_BANDS)
        img = imgcol.median()

        # add nightlights, latitude, and longitude bands
        img = ee_utils.add_latlon(img)
        img = img.addBands(ee_utils.composite_nl(year))

        fname = f'{country}_{year}_{i:02d}'
        tasks[(export_folder, country, year, i)] = ee_utils.get_array_patches(
            img=img, scale=SCALE, ksize=EXPORT_TILE_RADIUS,
            points=fc, export='drive',
            prefix=export_folder, fname=fname,
            bucket=None)
    return tasks

In [22]:
dhs_df = pd.read_csv(DHS_CSV_PATH, float_precision='high', index_col=False)
dhs_surveys = list(dhs_df.groupby(['country', 'year']).groups.keys())
dhs_df = dhs_df.head(5)
dhs_df.head(20)

,country,survey_start_year,year,lat,lon,GID_1,GID_2,households,rural,iwi
0,angola,2006,2006,-12.169283,13.859255,AGO.2,AGO.2.9,20,0,68.096300
1,angola,2006,2006,-12.361700,14.778201,AGO.2,AGO.2.2,19,1,10.609053
2,angola,2006,2006,-10.827451,14.342114,AGO.7,AGO.7.1,25,1,29.973928
3,angola,2006,2006,-9.991311,14.898295,AGO.7,AGO.7.5,1,1,7.704135
4,angola,2006,2006,-11.518893,14.750437,AGO.7,AGO.7.10,24,1,13.371826


In [23]:
dhs_surveys=dhs_surveys[0:5]
dhs_surveys

[('angola', 2006),
 ('angola', 2007),
 ('angola', 2010),
 ('angola', 2011),
 ('angola', 2015)]

In [24]:
country = list(dhs_surveys[0])[0]
year = list(dhs_surveys[0])[1]
country,year, DHS_EXPORT_FOLDER

('angola', 2006, 'dhs_tfrecords_raw')

In [ ]:

tasks = {}

for country, year in dhs_surveys:
    new_tasks = export_images(
        df=dhs_df, country=country, year=year,
        export_folder=DHS_EXPORT_FOLDER, chunk_size=CHUNK_SIZE)
    tasks.update(new_tasks)

Check on the status of each export task at [https://code.earthengine.google.com/](https://code.earthengine.google.com/), or run the following cell which checks every minute. Once all tasks have completed, download the DHS TFRecord files to `data/dhs_tfrecords_raw/`.

In [ ]:
ee_utils.wait_on_tasks(tasks, poll_interval=60)

  0%|          | 0/5 [00:00<?, ?it/s]

Task ('dhs_tfrecords_raw', 'angola', 2006, 0) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw', 'angola', 2006, 1) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw', 'angola', 2006, 2) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw', 'angola', 2006, 3) finished in 0 min with state: COMPLETED
Task ('dhs_tfrecords_raw', 'angola', 2006, 4) finished in 0 min with state: COMPLETED


## Visualize an exported image

In [ ]:
'''
 ----- The .tfrecord image has the following information -----

Available feature keys: ['year', 'GREEN', 'GID_2', 'rural', 'TEMP1', 'NIGHTLIGHTS', 'LON', 'RED', 'SWIR2', 'system:index', 'BLUE', 'iwi', 'country', 'survey_start_year', 'lon', 'households', 'lat', 'GID_1', 'SWIR1', 'NIR', 'LAT']
year: float_list with 1 elements
GREEN: float_list with 65025 elements
GID_2: bytes_list with 1 elements
rural: float_list with 1 elements
TEMP1: float_list with 65025 elements
NIGHTLIGHTS: float_list with 65025 elements
LON: float_list with 65025 elements
RED: float_list with 65025 elements
SWIR2: float_list with 65025 elements
system:index: bytes_list with 1 elements
BLUE: float_list with 65025 elements
iwi: float_list with 1 elements
country: bytes_list with 1 elements
survey_start_year: float_list with 1 elements
lon: float_list with 1 elements
households: float_list with 1 elements
lat: float_list with 1 elements
GID_1: bytes_list with 1 elements
SWIR1: float_list with 65025 elements
NIR: float_list with 65025 elements
LAT: float_list with 65025 elements
'''

# Path to the TFRecord file
tfrecord_path = './dhs_tfrecords_raw/angola_2006_01.tfrecord/angola_2006_01.tfrecord'

# Define the bands
image_bands = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2', 'TEMP1', 'NIGHTLIGHTS']
scalar_features = ['year', 'rural', 'iwi', 'survey_start_year', 'lon', 'lat', 'households']
string_features = ['GID_1', 'GID_2', 'country', 'system:index']

# feature description
feature_description = {}
for band in image_bands:
    feature_description[band] = tf.io.FixedLenFeature([65025], tf.float32)
for feature in scalar_features:
    feature_description[feature] = tf.io.FixedLenFeature([1], tf.float32)
for feature in string_features:
    feature_description[feature] = tf.io.FixedLenFeature([], tf.string)

# Parse function
def _parse_function(ds_set):
    return tf.io.parse_single_example(ds_set, feature_description)

# Create the dataset
dataset = tf.data.TFRecordDataset([tfrecord_path]).map(_parse_function)

# first image:
for example in dataset.take(1):
    # Print some metadata
    print(f"Country: {example['country'].numpy().decode('utf-8')}")
    print(f"Year: {int(example['year'].numpy()[0])}")
    print(f"Location: lat={float(example['lat'].numpy()[0])}, lon={float(example['lon'].numpy()[0])}")
    print(f'Shape of the image: {example["BLUE"].numpy().shape}')


    # Visualization
    plt.figure(figsize=(20, 15))

    # RGB True Color Image
    plt.subplot(2, 3, 1)
    rgb = np.stack([
        np.reshape(example['RED'].numpy(), (255, 255)),
        np.reshape(example['GREEN'].numpy(), (255, 255)),
        np.reshape(example['BLUE'].numpy(), (255, 255))
    ], axis=-1)

    # Normalize for visualization
    rgb_norm = (rgb - np.min(rgb)) / (np.max(rgb) - np.min(rgb))
    plt.imshow(rgb_norm)
    plt.title('RGB True Color')
    plt.axis('off')

    # False Color (NIR, RED, GREEN)
    plt.subplot(2, 3, 2)
    false_color = np.stack([
        np.reshape(example['NIR'].numpy(), (255, 255)),
        np.reshape(example['RED'].numpy(), (255, 255)),
        np.reshape(example['GREEN'].numpy(), (255, 255))
    ], axis=-1)

    # Normalize
    false_color_norm = (false_color - np.min(false_color)) / (np.max(false_color) - np.min(false_color))
    plt.imshow(false_color_norm)
    plt.title('False Color (NIR-RED-GREEN)')
    plt.axis('off')

    # SWIR Composite (SWIR1, NIR, RED)
    plt.subplot(2, 3, 3)
    swir_composite = np.stack([
        np.reshape(example['SWIR1'].numpy(), (255, 255)),
        np.reshape(example['NIR'].numpy(), (255, 255)),
        np.reshape(example['RED'].numpy(), (255, 255))
    ], axis=-1)

    # Normalize
    swir_norm = (swir_composite - np.min(swir_composite)) / (np.max(swir_composite) - np.min(swir_composite))
    plt.imshow(swir_norm)
    plt.title('SWIR Composite (SWIR1-NIR-RED)')
    plt.axis('off')

    # Temperature
    plt.subplot(2, 3, 4)
    temp = np.reshape(example['TEMP1'].numpy(), (255, 255))
    plt.imshow(temp, cmap='inferno')
    plt.colorbar(label='Temperature')
    plt.title('Temperature')
    plt.axis('off')

    # Night Lights
    plt.subplot(2, 3, 5)
    night = np.reshape(example['NIGHTLIGHTS'].numpy(), (255, 255))
    plt.imshow(night, cmap='viridis')
    plt.colorbar(label='Night Lights')
    plt.title('Night Lights')
    plt.axis('off')

    # Add a title for the entire figure
    plt.suptitle(f"Satellite Imagery for {example['country'].numpy().decode('utf-8')} ({int(example['year'].numpy()[0])})",
                fontsize=16)

    plt.tight_layout()
    plt.savefig('satellite_visualization.png', dpi=300)
    plt.show()

    # # Save the images
    # bands_to_save = ['RED', 'GREEN', 'BLUE', 'NIR', 'SWIR1', 'SWIR2']
    # for band in bands_to_save:
    #     plt.figure(figsize=(8, 8))
    #     band_data = np.reshape(example[band].numpy(), (255, 255))
    #     plt.imshow(band_data, cmap='gray')
    #     plt.title(f'{band} Band')
    #     plt.colorbar()
    #     plt.axis('off')
    #     plt.tight_layout()
    #     plt.close()

Output hidden; open in https://colab.research.google.com to view.